<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Projects/blob/main/Mask_NoMask_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install roboflow
import os
import glob
import pandas as pd
from roboflow import Roboflow
import xml.etree.ElementTree as ET

# Data

In [2]:
rf = Roboflow(api_key="Tqkp5uu6f4yAOuShs3oP")
project = rf.workspace("joseph-nelson").project("mask-wearing")
dataset = project.version(4).download("tensorflow")
os.rename('/content/Mask-Wearing-4', '/content/data')

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Mask-Wearing-4 in tensorflow:: 100%|██████████| 154/154 [00:00<00:00, 793.86it/s]


## XML to CSV
Convert annotation files(xml format) to a csv file.
(for manually annotated imges)


In [3]:
def xml_to_csv(path):
    xml_list = []
    for xml_file in glob.glob(path + '/*.xml'):
        tree = ET.parse(xml_file)
        root = tree.getroot()
        for member in root.findall('object'):
            value = (root.find('filename').text,
                     int(root.find('size')[0].text),
                     int(root.find('size')[1].text),
                     member[0].text,
                     int(member[4][0].text),
                     int(member[4][1].text),
                     int(member[4][2].text),
                     int(member[4][3].text))
            
            xml_list.append(value)
    column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
    xml_df = pd.DataFrame(xml_list, columns=column_name)
    return xml_df

In [4]:
images_path = '/content/'

xml_df = xml_to_csv(images_path)
xml_df.to_csv('/content/annotations.csv', index=None)    

## Label Map

Used in TensorFlow Object Detection API

In [5]:
labelmap_file = '''item {
    id: 1
    name: 'mask'
}
item {
    id: 2
    name: 'no-mask'
}'''

with open('/content/data/labelmap.pbtxt', 'w+') as f:
  f.write(labelmap_file)
f.close()

## Transfer Dataset to TFrecords

### Install Object Detection API


In [6]:
import pathlib
# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3634, done.
remote: Counting objects: 100% (3634/3634), done.
remote: Compressing objects: 100% (3030/3030), done.
remote: Total 3634 (delta 960), reused 1536 (delta 550), pack-reused 0
Receiving objects: 100% (3634/3634), 47.37 MiB | 18.46 MiB/s, done.
Resolving deltas: 100% (960/960), done.


In [7]:
# Install the Object Detection API
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Processing /content/models/research
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1696474 sha256=9f5a455a0f6a48fa557644bd4d81fd27b2fe23a6dd8a385750711bd621076beb
  Stored in directory: /tmp/pip-ephem-wheel-cache-znhflghj/wheels/7d/96/c1/072a751379735e8dfdada1def1c62a89afb3cc45654fd6fd28
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78544 sha256=43d93c1c06b9a5f1ef560e8fb21bae60be799daadeda31dee973dfdf6f98956a
  Stored in directory: /root/.cache/pip/wheels/07/35/78/e9004fa30578734db7f10e7a211605f3f0778d2bdde38a239d
  Created wheel for avro-python3: filename=avro_python3-1.10.2-py3-none-any.whl size=44009 sha256=66c8a57df5d5022acd6b0647377b2ea8b94c655cb14a6215bb4a4ad9c70734fe
  Stored in directory: /root/.cache/pip/wheels/bb/73/e9/d273421f5723c4bf544dcf9eb097bda94421ef8d3252699f0a
  Created wheel for docopt: filename=docopt-

  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


In [8]:
import io
import argparse
import collections
from PIL import Image
import tensorflow as tf

from object_detection.utils import dataset_util
from object_detection.utils import label_map_util

def split(df, group):
	data = collections.namedtuple('data', ['filename', 'object'])
	gb = df.groupby(group)
	return [data(filename, gb.get_group(x)) for filename, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path, category_idx):
	with tf.io.gfile.GFile(os.path.join(path, '{}'.format(group.filename)), 'rb') as fid:
		encoded_jpg = fid.read()
	encoded_jpg_io = io.BytesIO(encoded_jpg)
	image = Image.open(encoded_jpg_io)
	width, height = image.size

	filename = group.filename.encode('utf8')
	image_format = b'jpg'
	xmins = []
	xmaxs = []
	ymins = []
	ymaxs = []
	classes_text = []
	classes = []

	for index, row in group.object.iterrows():
		xmins.append(float(row['xmin']) / width)
		xmaxs.append(float(row['xmax']) / width)
		ymins.append(float(row['ymin']) / height)
		ymaxs.append(float(row['ymax']) / height)
		classes_text.append(row['class'].encode('utf8'))
		classes.append(category_idx[row['class']])

	tf_example = tf.train.Example(features=tf.train.Features(feature={
		'image/height': dataset_util.int64_feature(height),
		'image/width': dataset_util.int64_feature(width),
		'image/filename': dataset_util.bytes_feature(filename),
		'image/source_id': dataset_util.bytes_feature(filename),
		'image/encoded': dataset_util.bytes_feature(encoded_jpg),
		'image/format': dataset_util.bytes_feature(image_format),
		'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
		'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
		'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
		'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
		'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
		'image/object/class/label': dataset_util.int64_list_feature(classes),
	}))
	return tf_example

In [9]:
for f_name in ['train', 'test', 'valid']:

  img_path = f'/content/data/{f_name}'
  csv_path = f'/content/data/{f_name}/_annotations.csv'
  tfrecord_path = f'/content/data/{f_name}/{f_name}.tfrecord'
  labelmap_path  = '/content/data/labelmap.pbtxt'

  label_map_dict = label_map_util.get_label_map_dict(labelmap_path)
  writer = tf.io.TFRecordWriter(tfrecord_path)

  examples = pd.read_csv(csv_path)
  grouped = split(examples, 'filename')
  for group in grouped:
    tf_example = create_tf_example(group, img_path, label_map_dict)
    writer.write(tf_example.SerializeToString())

  writer.close()
  print(f'Successfully created the TFRecords: {tfrecord_path}')

Successfully created the TFRecords: /content/data/train/train.tfrecord
Successfully created the TFRecords: /content/data/test/test.tfrecord
Successfully created the TFRecords: /content/data/valid/valid.tfrecord


#Model

## Install a model from the model garden
Choose from a large selection of object detection models in the [TensorFlow 2 Detection Model Zoo](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md).

In [10]:
MODEL = "faster_rcnn_resnet50_v1_640x640_coco17_tpu-8"

%cd /content/models/research/object_detection
!mkdir pre-trained-models
%cd pre-trained-models
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/{MODEL}.tar.gz
!tar -xf {MODEL}.tar.gz
!rm {MODEL}.tar.gz

/content/models/research/object_detection
/content/models/research/object_detection/pre-trained-models
--2022-12-29 19:00:41--  http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 74.125.195.128, 2607:f8b0:400e:c09::80
Connecting to download.tensorflow.org (download.tensorflow.org)|74.125.195.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 211996178 (202M) [application/x-tar]
Saving to: ‘faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz’

faster_rcnn_resnet5 100%[===================>] 202.17M   143MB/s    in 1.4s    

2022-12-29 19:00:43 (143 MB/s) - ‘faster_rcnn_resnet50_v1_640x640_coco17_tpu-8.tar.gz’ saved [211996178/211996178]



## Edit pipeline.config
The pipeline.config file is in protocol buffer format, and we can use the config_util to change the settings we need.

In [11]:
from object_detection.utils import config_util


from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

In [13]:
pipeline_config = f"./{MODEL}/pipeline.config"
model_dir = f"./{MODEL}/checkpoint"

config = config_util.get_configs_from_pipeline_file(pipeline_config)

model_config = config['model']
model_config.faster_rcnn.num_classes = 2

train_config = config['train_config']

#Starting checkpoin location
train_config.fine_tune_checkpoint = f"/content/models/research/object_detection/pre-trained-models/{MODEL}/checkpoint/ckpt-0"

# GPU memory limits batch size
train_config.batch_size = 32
train_config.use_bfloat16 = False


#...................$$$$$$$$$$$$$$$.................
# some models use "fine_tune" rather than "detection" for fine-tuning
train_config.fine_tune_checkpoint_type = "detection"
# detection      ===> use all the layers as pretrained layers
# classification ===> use only the classification layer as pretrained and initialize the weights randomly


# set auto-augmentation
# del train_config.data_augmentation_options[:]

# a1=train_config.data_augmentation_options.add()
# a1.autoaugment_image.policy_name = "v2"

# training dataset
train_input_config = config['train_input_config']
train_input_config.label_map_path = "/content/data/labelmap.pbtxt"
train_input_config.tf_record_input_reader.input_path[:] = ["/content/data/train/*.tfrecord",
                                                           "/content/data/eval/*.tfrecord"]  # have little data, so we use eval as train data
# evaluation dataset                                                           
eval_input_config = config['eval_input_config']
eval_input_config.label_map_path = "/content/data/labelmap.pbtxt"
eval_input_config.tf_record_input_reader.input_path[:] = ["/content/data/test/*.tfrecord"]

# save pipeline.config file
pipeline_proto = config_util.create_pipeline_proto_from_configs(config)
config_util.save_pipeline_config(pipeline_config=pipeline_proto, directory=f'/content/models/research/object_detection/pre-trained-models/{MODEL}')

Check the contents of the pipeline.config file:


In [14]:
# !cat /content/models/research/object_detection/pre-trained-models/{MODEL}/pipeline.config

## Training

In [15]:
!mkdir /content/data/train_process/

In [20]:
%cd /content/models/research
!python object_detection/model_main_tf2.py \
    --num_train_steps=1000 \
    --pipeline_config_path=/content/models/research/object_detection/pre-trained-models/{MODEL}/pipeline.config \
    --model_dir=/content/data/train_process/ \
    --alsologtostderr
    #  &> /content/data/train.log

/content/models/research
2022-12-29 19:42:15.289346: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/cv2/../../lib64:/usr/lib64-nvidia
2022-12-29 19:42:15.289685: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib/python3.8/dist-packages/cv2/../../lib64:/usr/lib64-nvidia
2022-12-29 19:42:15.289715: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2022-12-29 19:42:21.486844: W te

In [19]:
!tail -n 10 /content/data/train.log

      return multilevel_roi_align(images,
    File "/usr/local/lib/python3.8/dist-packages/object_detection/utils/spatial_transform_ops.py", line 358, in multilevel_roi_align
      flattened_feature_values = _gather_valid_indices(flattened_features,
    File "/usr/local/lib/python3.8/dist-packages/object_detection/utils/spatial_transform_ops.py", line 275, in _gather_valid_indices
      return tf.gather(padded_tensor, indices + 1)
Node: 'MultiLevelMatMulCropAndResize/MultiLevelRoIAlign/GatherV2_1'
OOM when allocating tensor with shape[7526400,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node MultiLevelMatMulCropAndResize/MultiLevelRoIAlign/GatherV2_1}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference__dummy_computation_fn_99545]


## Evaluaion

In [ ]:
!python object_detection/model_main_tf2.py \
    --pipeline_config_path=/content/models/research/object_detection/pre-trained-models/{MODEL}/pipeline.config \
    --model_dir=/content/data/train_process/ \
    --checkpoint_dir=/content/data/train_process/ \
    --eval_timeout=30 \
    --alsologtostderr

#checkpoint_dir:  Only different to select it as Testing not training the

In [ ]:
#Copy the piplie.config for later use
!cp /content/models/research/object_detection/pre-trained-models/{MODEL}/pipeline.config /content/training/.

In [ ]:
#Export trained model
!python object_detection/exporter_main_v2.py \
    --input_type image_tensor \
    --pipeline_config_path=/kaggle/working/models/research/object_detection/pre-trained-models/{MODEL}/pipeline.config \
    --trained_checkpoint_dir /kaggle/working/training/ \
    --output_directory /kaggle/working/exported-models/effdet_d2